<a href="https://colab.research.google.com/github/dfvnunes/Pilotos-F1/blob/main/Pilotos_F1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import pandas as pd
import re

def clean_brackets(val: str) -> str:
    """
    Remove everything between square brackets, preserving the rest.
    E.g. "17[1]" -> "17".
    """
    return re.sub(r"\[.*?\]", "", str(val)).strip()

def extract_parentheses(val: str) -> str:
    """
    Extract the content inside the last pair of parentheses.
    E.g. "107,64 (140,14) [1/7]" -> "140,14"; "255 (274)" -> "274".
    """
    matches = re.findall(r"\(([^)]+)\)", str(val))
    return matches[-1] if matches else clean_brackets(val)

def extract_years(season_str):

    """
    Extracts every individual year from a seasons string, going through
    all the ranges (e.g.:'1989-2001') and lists separated wuth commas.
    Returns a set of years.
    """

    years = set()
    # Remove aspas e divide por vírgulas
    for part in str(season_str).replace('"', '').split(','):
        part = part.strip()
        if '-' in part:
            start, end = part.split('-')
            try:
                start, end = int(start), int(end)
                # Adiciona todos os anos no intervalo inclusive
                years.update(range(start, end + 1))
            except ValueError:
                continue
        else:
            try:
                years.add(int(part))
            except ValueError:
                continue
    return years

def dominant_decade(season_str):

    """
    Based on the seasons string, calculates the dominant decade,
    the one where each driver raced for most years.
    Returns the first year of each decade (1990, 1970).
    """

    years = extract_years(season_str)
    if not years:
        return None

    # Counts how many years in each decade
    decade_counts = {}
    for y in years:
        dec = (y // 10) * 10
        decade_counts[dec] = decade_counts.get(dec, 0) + 1

      # Selects the decade with more years (in case of a 50/50
      # split the most recent is selected)
    dominant = max(decade_counts.items(), key=lambda x: (x[1], x[0]))[0]
    return dominant

# 1. Load the original Excel file
input_path = 'Pilotos F1.xlsx'
df = pd.read_excel(input_path, sheet_name='Pilotos F1')

# 2. Clean all columns except "Pontos": remove bracketed notes
cols_to_clean = [c for c in df.columns if c != 'Pontos']
for col in cols_to_clean:
    df[col] = df[col].apply(clean_brackets)

# 3. Clean "Pontos" column: keep only the value in parentheses (or main value if none)
df['Pontos'] = df['Pontos'].apply(extract_parentheses)

# 4. Add column 'Década_Predominante'
df['Década_Predominante'] = df['Temporadas'].apply(dominant_decade)

# 5. Save data to a new Excel file
output_path = 'Pilotos-F1_limpo.xlsx'
df.to_excel(output_path, index=False)

print(f"Limpeza concluída: {len(df)} linhas processadas. Ficheiro guardado em '{output_path}'.")


Limpeza concluída: 873 linhas processadas. Ficheiro guardado em 'Pilotos-F1_limpo.xlsx'.
